In [1]:
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras import backend as K

from MD_AE_model import *
import myplot

import h5py
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## import data
folder = 'C:/Users/tracy/OneDrive - Imperial College London/PhD/Code_md-ae/Hierarchical_2_1__2022_04_01__11_59_23/'
filename = folder + 'results.h5'
hf = h5py.File(filename,'r')
u_test = np.array(hf.get('u_test'))
hist_train = np.array(hf.get('hist_train'))
u_avg = np.array(hf.get('u_avg'))
latent_test = np.array(hf.get('latent_test'))
hf.close()

In [5]:
filename = folder + 'Model_param.h5'
hf = h5py.File(filename,'r')
Ny = hf.get('Ny')[()]
Nz = hf.get('Nz')[()]
Nu = hf.get('Nu')[()]
features_layers = np.array(hf.get('features_layers')).tolist()
no_of_modes = int(hf.get('no_of_modes')[()])
latent_dim = int(hf.get('latent_dim')[()])
resize_meth = hf.get('resize_meth')[()].decode()
filter_window = np.array(hf.get('filter_window')).tolist()
act_fct = hf.get('act_fct')[()].decode()
batch_norm = hf.get('batch_norm')[()]
drop_rate = hf.get('drop_rate')[()]
lmb = hf.get('lmb')[()]
idx_unshuffle = np.array(hf.get('idx_unshuffle'))
hf.close()

Nx = [Ny,Nz]
previous_dim = []
subnets = []
for _ in range(no_of_modes):
    subnets.extend([hierarchicalAE_sub(Nx=Nx,Nu=Nu,previous_dim=previous_dim,features_layers=features_layers,latent_dim=latent_dim,filter_window=filter_window,act_fct=act_fct,drop_rate=drop_rate,lmb=lmb)])
    previous_dim.extend([latent_dim])



# load_which = [0,1] # start from 0
# for net in load_which:
#     weights = folder + 'subnet' + str(net+1) + '.h5'
#     subnets[net].load_weights(weights)

# ## load model
# filename = folder + 'subnet1'
# subnet1 = tf.keras.models.load_model(filename)
# filename = folder + 'subnet2'
# subnet2 = tf.keras.models.load_model(filename)

call
call


ValueError: Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.

In [8]:
weights = folder + 'subnet1.h5'
out = subnets[0].build(subnets[0].full_input)
subnets[0].load_weights(weights)

TypeError: Error converting shape to a TensorShape: Dimension value must be integer or None or have an __index__ method, got value '<KerasTensor: shape=(None, 21, 24, 2) dtype=float32 (created by layer 'input_15')>' with type '<class 'keras.engine.keras_tensor.KerasTensor'>'.

In [ ]:
## latent space
z1 = subnets[0].encoder.predict(u_test)
z2 = subnets[1].encoder.predict(u_test)
plt.figure()
plt.plot(z1,label='1')
plt.plot(z2,label='2')
plt.legend()
plt.title('latent space')
plt.show()

In [ ]:
# autoencoder results
y_test = subnets[0].predict([u_test])
# y_test = subnet2.predict([u_test,z1])

myplot.plot_ae_results(u_test,y_test,u_avg,'mae')

In [ ]:
## modes
mode1 = subnets[0].predict([u_test])
mode2 = subnets[1].predict([u_test,z1]) - mode1
print(mode1.shape)

In [ ]:
## time averaged modes
fig,ax = plt.subplots(2,2)

plt.suptitle('time averaged autoencoder modes')
mode1_v = ax[0,0].imshow(np.mean(mode1,axis=0)[:,:,0],'jet')
ax[0,0].set_title('Mode 1, v')
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])
fig.colorbar(mode1_v,ax=ax[0,0])

mode1_w = ax[0,1].imshow(np.mean(mode1,axis=0)[:,:,1],'jet')
ax[0,1].set_title('Mode 1, w')
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
fig.colorbar(mode1_w,ax=ax[0,1])

mode2_v = ax[1,0].imshow(np.mean(mode2,axis=0)[:,:,0],'jet')
ax[1,0].set_title('Mode 2, v')
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
fig.colorbar(mode2_v,ax=ax[1,0])

mode2_w = ax[1,1].imshow(np.mean(mode2,axis=0)[:,:,1],'jet')
ax[1,1].set_title('Mode 2, w')
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
fig.colorbar(mode2_w,ax=ax[1,1])

plt.show()

In [ ]:
## plt modes
fig,ax = plt.subplots(2,2)
time = 450

plt.suptitle('modes at snapshot '+str(time))
mode1_v = ax[0,0].imshow(mode1[time,:,:,0],'jet')
ax[0,0].set_title('Mode 1, v')
ax[0,0].set_xticks([])
ax[0,0].set_yticks([])
fig.colorbar(mode1_v,ax=ax[0,0])

mode1_w = ax[0,1].imshow(mode1[time,:,:,1],'jet')
ax[0,1].set_title('Mode 1, w')
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
fig.colorbar(mode1_w,ax=ax[0,1])

mode2_v = ax[1,0].imshow(mode2[time,:,:,0],'jet')
ax[1,0].set_title('Mode 2, v')
ax[1,0].set_xticks([])
ax[1,0].set_yticks([])
fig.colorbar(mode2_v,ax=ax[1,0])

mode2_w = ax[1,1].imshow(mode2[time,:,:,1],'jet')
ax[1,1].set_title('Mode 2, w')
ax[1,1].set_xticks([])
ax[1,1].set_yticks([])
fig.colorbar(mode2_w,ax=ax[1,1])

plt.show()

In [ ]:
idx = np.where(hist_train == -1)[0]
plt.figure()
plt.plot(hist_train[:idx[0]],label='subnet 1')
plt.plot(hist_train[idx[0]+1:],label='subnet 2') 
plt.legend()
plt.show()